In [2]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt 
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier 
from sklearn.tree import DecisionTreeClassifier 
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier 
from xgboost import XGBClassifier
from catboost import CatBoostClassifier 



# Read The Data

In [3]:
df = pd.read_csv('Data\BreasetConser.csv')

**shows the first 5 columns**  

In [4]:
df.head(5)

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status
0,42,FEMALE,0.95256,2.15000,0.007972,-0.048340,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,2018-05-20,2018-08-26,Alive
1,54,FEMALE,0.00000,1.38020,-0.498030,-0.507320,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,2018-04-26,2019-01-25,Dead
2,63,FEMALE,-0.52303,1.76400,-0.370190,0.010815,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Lumpectomy,2018-08-24,2020-04-08,Alive
3,78,FEMALE,-0.87618,0.12943,-0.370380,0.132190,I,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,2018-11-16,2020-07-28,Alive
4,42,FEMALE,0.22611,1.74910,-0.543970,-0.390210,II,Infiltrating Ductal Carcinoma,Positive,Positive,Positive,Lumpectomy,2018-12-12,2019-01-05,Alive


**Preparing X and Y** 

**Categories columns**

In [6]:
print("Categories in 'gender' variable:  ",end=" ")
print(df['Gender'].unique())
print("Categories in 'Tumor stage' variable:  ",end=" ")
print(df['Tumour_Stage'].unique())
print("Categories in 'Histology' variable:  ",end=" ")
print(df['Histology'].unique())
print("Categories in 'ER status' variable:  ",end=" ")
print(df['ER status'].unique())
print("Categories in 'PR status' variable:  ",end=" ")
print(df['PR status'].unique())
print("Categories in 'HER2 status' variable:  ",end=" ")
print(df['HER2 status'].unique())
print("Categories in 'Surgery Type' variable:  ",end=" ")
print(df['Surgery_type'].unique())

Categories in 'gender' variable:   ['FEMALE' 'MALE']
Categories in 'Tumor stage' variable:   ['II' 'I' 'III']
Categories in 'Histology' variable:   ['Infiltrating Ductal Carcinoma' 'Infiltrating Lobular Carcinoma'
 'Mucinous Carcinoma']
Categories in 'ER status' variable:   ['Positive']
Categories in 'PR status' variable:   ['Positive']
Categories in 'HER2 status' variable:   ['Negative' 'Positive']
Categories in 'Surgery Type' variable:   ['Other' 'Lumpectomy' 'Modified Radical Mastectomy' 'Simple Mastectomy']


In [30]:
df['Date_of_Last_Visit']=pd.to_datetime(df['Date_of_Last_Visit'])
df['Date_of_Surgery']= pd.to_datetime(df['Date_of_Surgery'])

In [9]:
df.head()

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,ER status,PR status,HER2 status,Surgery_type,Patient_Status,year_Visit,month_Visit,day_Visit,year_Surgery,month_Surgery,day_Surgery
0,42,FEMALE,0.95256,2.15000,0.007972,-0.048340,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,Alive,2018,8,26,2018,5,20
1,54,FEMALE,0.00000,1.38020,-0.498030,-0.507320,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,Dead,2019,1,25,2018,4,26
2,63,FEMALE,-0.52303,1.76400,-0.370190,0.010815,II,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Lumpectomy,Alive,2020,4,8,2018,8,24
3,78,FEMALE,-0.87618,0.12943,-0.370380,0.132190,I,Infiltrating Ductal Carcinoma,Positive,Positive,Negative,Other,Alive,2020,7,28,2018,11,16
4,42,FEMALE,0.22611,1.74910,-0.543970,-0.390210,II,Infiltrating Ductal Carcinoma,Positive,Positive,Positive,Lumpectomy,Alive,2019,1,5,2018,12,12


In [13]:
Gender_feature = X['Gender']
Gender_feature

0      FEMALE
1      FEMALE
2      FEMALE
3      FEMALE
4      FEMALE
        ...  
312    FEMALE
313    FEMALE
314    FEMALE
315    FEMALE
316    FEMALE
Name: Gender, Length: 317, dtype: object

In [53]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler,OrdinalEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer

In [37]:
df_processing = df.drop(['ER status','PR status'],axis=1)

In [38]:
df_processing.sample(1)

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,HER2 status,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status
137,45,FEMALE,0.42538,1.2628,-0.71996,-1.1641,II,Infiltrating Ductal Carcinoma,Negative,Simple Mastectomy,2018-01-16,2020-02-10,Alive


In [39]:
#Encoder Tumor stage,histology and surgery type with ordinalencoder 

ordinal_encoder = OrdinalEncoder(categories=[['I','II','III'],
                                             ['Infiltrating Ductal Carcinoma','Infiltrating Lobular Carcinoma','Mucinous Carcinoma'],
                                             ['Other','Lumpectomy','Modified Radical Mastectomy','Simple Mastectomy']])

df_processing[['Tumour_Stage','Histology','Surgery_type']] = ordinal_encoder.fit_transform(df_processing[['Tumour_Stage','Histology','Surgery_type']])

In [40]:
df_processing.sample()

,Age,Gender,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,HER2 status,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status
174,79,FEMALE,0.4794,2.0559,-0.53136,-0.18848,0.0,0.0,Positive,1.0,2019-11-21,2021-02-19,Dead


In [41]:
#Encoder Gender and HER2 stutus with getdummies (binary encoding)
df_processing = pd.get_dummies(data=df_processing,columns=['Gender','HER2 status'],drop_first=True)

In [42]:
df_processing.head()

,Age,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,Surgery_type,Date_of_Surgery,Date_of_Last_Visit,Patient_Status,Gender_MALE,HER2 status_Positive
0,42,0.95256,2.15000,0.007972,-0.048340,1.0,0.0,0.0,2018-05-20,2018-08-26,Alive,False,False
1,54,0.00000,1.38020,-0.498030,-0.507320,1.0,0.0,0.0,2018-04-26,2019-01-25,Dead,False,False
2,63,-0.52303,1.76400,-0.370190,0.010815,1.0,0.0,1.0,2018-08-24,2020-04-08,Alive,False,False
3,78,-0.87618,0.12943,-0.370380,0.132190,0.0,0.0,0.0,2018-11-16,2020-07-28,Alive,False,False
4,42,0.22611,1.74910,-0.543970,-0.390210,1.0,0.0,1.0,2018-12-12,2019-01-05,Alive,False,True


In [43]:
#extract day,monthm,year from Date_of_Last_Visit
df_processing['year_Visit'] = df_processing['Date_of_Last_Visit'].dt.year
df_processing['month_Visit'] = df_processing['Date_of_Last_Visit'].dt.month
df_processing['day_Visit'] = df_processing['Date_of_Last_Visit'].dt.day
df_processing.drop(columns='Date_of_Last_Visit',axis=1,inplace=True)

#extract day,month,year from Date_of_Last_Visit
df_processing['year_Surgery'] = df_processing['Date_of_Surgery'].dt.year
df_processing['month_Surgery'] = df_processing['Date_of_Surgery'].dt.month
df_processing['day_Surgery'] = df_processing['Date_of_Surgery'].dt.day
df_processing.drop(columns='Date_of_Surgery',axis=1,inplace=True)


In [44]:
df_processing.head()

,Age,Protein1,Protein2,Protein3,Protein4,Tumour_Stage,Histology,Surgery_type,Patient_Status,Gender_MALE,HER2 status_Positive,year_Visit,month_Visit,day_Visit,year_Surgery,month_Surgery,day_Surgery
0,42,0.95256,2.15000,0.007972,-0.048340,1.0,0.0,0.0,Alive,False,False,2018,8,26,2018,5,20
1,54,0.00000,1.38020,-0.498030,-0.507320,1.0,0.0,0.0,Dead,False,False,2019,1,25,2018,4,26
2,63,-0.52303,1.76400,-0.370190,0.010815,1.0,0.0,1.0,Alive,False,False,2020,4,8,2018,8,24
3,78,-0.87618,0.12943,-0.370380,0.132190,0.0,0.0,0.0,Alive,False,False,2020,7,28,2018,11,16
4,42,0.22611,1.74910,-0.543970,-0.390210,1.0,0.0,1.0,Alive,False,True,2019,1,5,2018,12,12


In [65]:
df_processing.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 317 entries, 0 to 316
Data columns (total 17 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Age                   317 non-null    int64  
 1   Protein1              317 non-null    float64
 2   Protein2              317 non-null    float64
 3   Protein3              317 non-null    float64
 4   Protein4              317 non-null    float64
 5   Tumour_Stage          317 non-null    float64
 6   Histology             317 non-null    float64
 7   Surgery_type          317 non-null    float64
 8   Patient_Status        317 non-null    object 
 9   Gender_MALE           317 non-null    int32  
 10  HER2 status_Positive  317 non-null    int32  
 11  year_Visit            317 non-null    int32  
 12  month_Visit           317 non-null    int32  
 13  day_Visit             317 non-null    int32  
 14  year_Surgery          317 non-null    int32  
 15  month_Surgery         3

In [64]:
df_processing = df_processing.astype({'Gender_MALE':int,'HER2 status_Positive':int})

In [66]:
numerical_features = [col for col in df_processing.columns if df_processing[col].dtype in['int32','float64','int64']]
numerical_features

['Age',
 'Protein1',
 'Protein2',
 'Protein3',
 'Protein4',
 'Tumour_Stage',
 'Histology',
 'Surgery_type',
 'Gender_MALE',
 'HER2 status_Positive',
 'year_Visit',
 'month_Visit',
 'day_Visit',
 'year_Surgery',
 'month_Surgery',
 'day_Surgery']

In [67]:
numric_processeur = Pipeline(steps=[
    ("imputer",SimpleImputer(strategy='mean')),
    ("scale",StandardScaler())
])
catg_processeur = Pipeline(steps=[
    ("imputer",SimpleImputer(strategy='most_frequient')),
    ("onehot",OneHotEncoder())
])

In [68]:
preprocessor = ColumnTransformer(
    transformers=[
        ('nmeric',numric_processeur,numerical_features)

    ]
)

In [69]:
X = df_processing.drop(columns='Patient_Status',axis=1)
y= df_processing['Patient_Status']

In [70]:
from sklearn.model_selection import train_test_split 

X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=32)

In [71]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.fit_transform(X_test)

In [72]:
label_encoder = LabelEncoder()
y_train = label_encoder.fit_transform(y_train)
y_test = label_encoder.fit_transform(y_test)

In [73]:
X_train.shape,X_test.shape

((253, 16), (64, 16))

In [80]:
def evaluate_model(true,predicted):
    mae = mean_absolute_error(true,predicted)
    mse = mean_squared_error(true,predicted)
    rmse = np.sqrt(mean_squared_error(true,predicted))
    r2_score_val = r2_score(true,predicted) 

    return mae,rmse,r2_score_val


In [74]:
models = {
    "LogisticRegression":LogisticRegression(),
    "RandomForestClassifier":RandomForestClassifier(),
    "AdaBoostClassifier":AdaBoostClassifier(),
    "CatBoostClassifier":CatBoostClassifier(verbose=False),
    "DecisionTreeClassifier":DecisionTreeClassifier(),
    'SVC':SVC(),
    'KNeighborsClassifier':KNeighborsClassifier(),
    'XGBClassifier':XGBClassifier()
}

In [86]:
model_list = []
r2_list = []
for i in range(len(list(models))): 
    model = list(models.values())[i]
    model.fit(X_train,y_train)

    #make prediction 

    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)

    #Evaluate Train and Test dataset: 
    
    model_train_mae , model_train_rmse ,model_train_r2 = evaluate_model(y_train,y_train_pred)
    model_test_mae , model_test_rmse ,model_test_r2 = evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i]) 
    model_list.append(list(models.keys())[i])

    print('Model performance for training set')
    print("- RMS Error : {:.2f}".format(model_train_rmse))
    print("- MAE Error : {:.2f}".format(model_train_mae))
    print("- R2 Score : {:.2f}".format(model_train_r2))

    print("------------------------------------------------")

    print('Model performance for test set')
    print("- RMS Error : {:.2f}".format(model_test_rmse))
    print("- MAE Error : {:.2f}".format(model_test_mae))
    print("- R2 Score : {:.2f}".format(model_test_r2))



    



LogisticRegression
Model performance for training set
- RMS Error : 0.44
- MAE Error : 0.19
- R2 Score : -0.20
------------------------------------------------
Model performance for test set
- RMS Error : 0.47
- MAE Error : 0.22
- R2 Score : -0.44
RandomForestClassifier
Model performance for training set
- RMS Error : 0.00
- MAE Error : 0.00
- R2 Score : 1.00
------------------------------------------------
Model performance for test set
- RMS Error : 0.43
- MAE Error : 0.19
- R2 Score : -0.23


e:\End to End ML\Breast Cancer Survival Prediction\venv\lib\site-packages\sklearn\ensemble\_weight_boosting.py:519: FutureWarning: The SAMME.R algorithm (the default) is deprecated and will be removed in 1.6. Use the SAMME algorithm to circumvent this warning.
  warnings.warn(


AdaBoostClassifier
Model performance for training set
- RMS Error : 0.28
- MAE Error : 0.08
- R2 Score : 0.50
------------------------------------------------
Model performance for test set
- RMS Error : 0.59
- MAE Error : 0.34
- R2 Score : -1.26
Learning rate set to 0.005729
0:	learn: 0.6893190	total: 13.7ms	remaining: 13.7s
1:	learn: 0.6863646	total: 19.6ms	remaining: 9.78s
2:	learn: 0.6833057	total: 29ms	remaining: 9.63s
3:	learn: 0.6805301	total: 34.4ms	remaining: 8.56s
4:	learn: 0.6777129	total: 44ms	remaining: 8.75s
5:	learn: 0.6744004	total: 50.4ms	remaining: 8.35s
6:	learn: 0.6715018	total: 60.3ms	remaining: 8.55s
7:	learn: 0.6680860	total: 67.1ms	remaining: 8.32s
8:	learn: 0.6650085	total: 76.9ms	remaining: 8.47s
9:	learn: 0.6612573	total: 82.4ms	remaining: 8.15s
10:	learn: 0.6584359	total: 90.7ms	remaining: 8.16s
11:	learn: 0.6557752	total: 96ms	remaining: 7.9s
12:	learn: 0.6535032	total: 99ms	remaining: 7.52s
13:	learn: 0.6509379	total: 105ms	remaining: 7.37s
14:	learn: 0.64